In [1]:
import os
import pandas as pd

In [2]:
data_dir = '/media/dmlab/My Passport/DATA/sec'
data_filepath = os.path.join(data_dir, '10K_parsed.csv')

In [3]:
%%time
df = pd.read_csv(data_filepath)
print(len(df))
df.head()

1429


,form_type,CIK,filing_date,period_of_report,texts_only,image_urls,files_url,form_url,Ticker,CompanyName,item1_business,item1a_risk,item7_mda
0,10-K,66740,2022-02-09,2021-12-31,mmm-202112310000066740falsefalse2021FYP10YP20Y...,[],https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,3M,Item 1. Business.3M Company was incorporated i...,"Item 1A, Risk Factors, of this document, and s...",Item 7 may involve risks and uncertainties tha...
1,10-K,66740,2021-02-04,2020-12-31,"Common Stock, Par Value $.01 Per Share--12-312...",[],https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,3M,"Item 1, Business Segments, provides an overvie...","Item 1A, Risk Factors, of this document, and s...",Item 7 may involve risks and uncertainties tha...
2,10-K,66740,2020-02-06,2019-12-31,"Common Stock, Par Value $.01 Per SharetrueP20Y...",[],https://www.sec.gov/Archives/edgar/data/66740/...,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,3M,"Item 1, Business Segments. Disclosures relatin...","Item 1A, Risk Factors, of this document, and s...",Item 7 may involve risks and uncertainties tha...
3,10-K,91142,2022-02-11,2021-12-31,aos-20211231false0000091142FY2021P3YP3YP5YP1YP...,['https://www.sec.gov/Archives/edgar/data/9114...,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...,AOS,A. O. Smith,Item 1.Business 3Item 1A.Risk Factors7Item 1B....,Item 1A.Risk Factors7Item 1B.Unresolved Staff ...,Item 7 of this Annual Report on Form 10-K.Our ...
4,10-K,91142,2021-02-12,2020-12-31,aos-20201231false0000091142FYAOS2020P3YP3YP5YP...,['https://www.sec.gov/Archives/edgar/data/9114...,https://www.sec.gov/Archives/edgar/data/91142/...,https://www.sec.gov/Archives/edgar/data/91142/...,AOS,A. O. Smith,Item 1.Business 3Item 1A.Risk Factors6Item 1B....,Item 1A.Risk Factors6Item 1B.Unresolved Staff ...,Item 7 of this Annual Report on Form 10-K.Our ...


In [4]:
original_len = len(df)

# FORM 10-K/A 삭제
df = df[df['item1_business']!='FORM 10-K/A!']
print('Dropped {}'.format(original_len - len(df)))

print('Final number of documents = {}'.format(len(df)))

Dropped 87
Final number of documents = 1342


In [5]:
for colname, origin_colname in [('len_item1', 'item1_business'), ('len_item1a', 'item1a_risk'), \
                                ('len_item7', 'item7_mda')]:
    df[colname] = df[origin_colname].apply(lambda x: len(x))
    print(origin_colname)
    print('min={}\nMedian={}\nmax={}'.format(df[colname].min(), df[colname].median(), df[colname].max()))
    print('Number of inappropriately processed documents: {} ({:.2f}%)\n'.format(\
         len(df[(df[origin_colname]=='Something went wrong!')]), 100*len(df[(df[origin_colname]=='Something went wrong!')])/len(df)))

item1_business
min=21
Median=39745.5
max=139163
Number of inappropriately processed documents: 55 (4.10%)

item1a_risk
min=21
Median=62702.0
max=156849
Number of inappropriately processed documents: 108 (8.05%)

item7_mda
min=21
Median=75065.0
max=173664
Number of inappropriately processed documents: 117 (8.72%)



잘 처리되지 못한 케이스 확인

In [14]:
colname = 'item1_business'
print(len(df[(df[colname]=='Something went wrong!')]['CompanyName'].unique()))
print(df[(df[colname]=='Something went wrong!')]['CompanyName'].unique())
df[(df[colname]=='Something went wrong!')]['form_url'].values[:5]

21
['Cardinal Health' 'Citigroup' 'Clorox' 'Edison International' 'GE'
 'Halliburton' 'Honeywell' 'Illumina' 'Intel' 'Intercontinental Exchange'
 "McDonald's" 'Moderna' 'Molina Healthcare' 'Morgan Stanley' 'Prudential'
 'PulteGroup' 'Rollins' 'Synchrony' 'Viatris' 'Vulcan Materials'
 'Weyerhaeuser']


array(['https://www.sec.gov/Archives/edgar/data/721371/000072137121000080/cah-20210630.htm',
       'https://www.sec.gov/Archives/edgar/data/721371/000072137120000089/a20q410k063020form10-k.htm',
       'https://www.sec.gov/Archives/edgar/data/831001/000083100122000036/c-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/831001/000083100121000042/c-20201231.htm',
       'https://www.sec.gov/Archives/edgar/data/831001/000083100120000031/c-12312019x10k.htm'],
      dtype=object)

In [15]:
colname = 'item1a_risk'
print(len(df[(df[colname]=='Something went wrong!')]['CompanyName'].unique()))
print(df[(df[colname]=='Something went wrong!')]['CompanyName'].unique())
df[(df[colname]=='Something went wrong!')]['form_url'].values[:5]

46
['Alexandria' 'American Airlines Group' 'Arista' 'AT&T' 'Booking Holdings'
 'Cardinal Health' 'Citigroup' 'Clorox' 'Constellation Brands' 'Costco'
 'DexCom' 'Digital Realty' 'Eastman' 'Edison International' 'Enphase'
 'Entergy' 'Etsy' 'Fortinet' 'GE' 'Global Payments' 'Halliburton'
 'Hartford (The)' 'Honeywell' 'Illumina' 'Intel'
 'Intercontinental Exchange' 'Intuitive Surgical' "McDonald's" 'Meta'
 'Moderna' 'Molina Healthcare' 'Morgan Stanley' 'Newmont' 'Pfizer'
 'Rollins' 'SBA Communications' 'Sempra Energy' 'State Street' 'Synchrony'
 'Truist' 'U.S. Bank' 'Viatris' 'Vulcan Materials' 'Wells Fargo'
 'West Pharmaceutical Services' 'Weyerhaeuser']


array(['https://www.sec.gov/Archives/edgar/data/1035443/000103544322000040/are-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/1035443/000103544320000042/a4q1910-k.htm',
       'https://www.sec.gov/Archives/edgar/data/4515/000000620122000026/aal-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/1596532/000159653222000026/anet-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/1596532/000159653220000024/anet20191231-10k.htm'],
      dtype=object)

In [16]:
colname = 'item7_mda'
print(len(df[(df[colname]=='Something went wrong!')]['CompanyName'].unique()))
print(df[(df[colname]=='Something went wrong!')]['CompanyName'].unique())
df[(df[colname]=='Something went wrong!')]['form_url'].values[:5]

45
['Allstate' 'American Electric Power' 'Bank of America'
 'Boston Properties' 'Capital One' 'Cardinal Health' 'Chevron' 'Chubb'
 'Citigroup' 'Clorox' 'Costco' 'Edison International' 'Fifth Third Bank'
 'Freeport-McMoRan' 'GE' 'Goldman Sachs' 'Halliburton' 'Hartford (The)'
 'Honeywell' 'Illumina' 'Intel' 'M&T Bank' "McDonald's" 'MetLife'
 'Molina Healthcare' 'Morgan Stanley' 'Mosaic' 'NiSource' 'Northern Trust'
 'PepsiCo' 'Progressive' 'Prudential' 'Quest Diagnostics' 'Regions'
 'State Street' 'SVB Financial' 'Synchrony' 'Travelers' 'Truist'
 'U.S. Bank' 'Universal Health Services' 'Viatris' 'Vulcan Materials'
 'Wells Fargo' 'Weyerhaeuser']


array(['https://www.sec.gov/Archives/edgar/data/899051/000089905122000015/all-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/899051/000089905121000011/all-20201231.htm',
       'https://www.sec.gov/Archives/edgar/data/899051/000089905120000013/allcorp-12311910xk.htm',
       'https://www.sec.gov/Archives/edgar/data/4904/000000490422000024/aep-20211231.htm',
       'https://www.sec.gov/Archives/edgar/data/4904/000000490421000010/aep-20201231.htm'],
      dtype=object)

In [9]:
# 잘 처리된 문서 개수
for origin_colname in ['item1_business', 'item1a_risk', 'item7_mda']:
    print('{},{} ({:.2f}%)'.format(origin_colname, len(df[(df[origin_colname]!='Something went wrong!')]), \
                              100*len(df[(df[origin_colname]!='Something went wrong!')])/len(df)))

item1_business,1287 (95.90%)
item1a_risk,1234 (91.95%)
item7_mda,1225 (91.28%)


In [ ]:
df[(df['len_item1']>21)].sort_values(by=['len_item1'])['form_url'].values[:5]

In [ ]:
content='''
GENERAL

As used in this Annual Report on Form 10-K, the “Company” refers, unless the context otherwise requires, to PGI Incorporated and its subsidiaries.  The Company’s executive offices are at 212 S. Central, St. Louis, Missouri, 63105, and its telephone number is (314) 512-8650.

The Company, a Florida corporation, was founded in 1958, and up until the mid 1990’s was in the business of building and selling homes, developing and selling home sites and selling undeveloped or partially developed tracts of land.  Over approximately the last 20 years, the Company’s business focus and emphasis changed substantially as it has concentrated its sales and marketing efforts almost exclusively on the disposition of its remaining real estate. This change was prompted by its continuing financial difficulties due to the principal and interest owed on its debt.

Presently, the most valuable remaining asset of the Company is an undeveloped parcel of 366 acres located in Hernando County, Florida. The Company also owns a number of scattered sites in Charlotte County, Florida (the “Charlotte Property”), which total approximately 60 acres, but most of these sites are subject to easements which markedly reduce their value, consist of wetlands of indeterminate value and/or are subject to obstacles to development and the sale of such property.  As of December 31, 2015, the Company also owned six single family lots and an approximate 7 acre parcel located in Citrus County, Florida.

As of December 31, 2015, the Company had no employees, and all services provided to the Company are through contract services.

MOST RECENT DEVELOPMENTS

The principal remaining real property asset of the Company is a 366 acre undeveloped parcel in Hernando County, Florida, which property is encumbered by secured creditor claims.

Until recently, the potential development and sale of this parcel was unclear due to the uncertainty regarding whether the Suncoast Parkway, which terminates on the south side of Route 98 opposite this property, would be extended.  The State of Florida, however, has indicated that it plans to proceed with the northward continuation of the Suncoast Parkway as part of the Suncoast Parkway Project 2.  The Florida Department of Transportation - Florida’s Turnpike Enterprise website for the Suncoast Parkway Project 2 provides preliminary plans and certain other information with respect to this proposed extension.  The Company believes that the State of Florida has begun the process of acquiring the required right-of-ways for this project and has a goal of acquiring them by December, 2016.  The State of Florida has expressed an interest in acquiring this 366 acre parcel, or a portion thereof, from the Company in connection with the Suncoast Parkway Project 2.  There is no agreement for the purchase of all or any portion of this 366 acre parcel by the State of Florida at this time.'''
len(content)